# MRE Assignment 3 - Digital Video Processing
In this assignment, you will use OpenCV and FFmpeg to implement very basic video editing functions. These tasks include:

1. Create a slide show (as a video) from images, and optionally create the slideshow as greyscale video.
2. Extract the audio track from a video file.
3. Replace the audio track in a video file.
4. Combine two or more videos into one video file.
5. Blend an image (fade-in/fade-out) with a video.
6. Blend two videos into one video (video collage).

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=164140#section-12).

If the cell below executes without error, you can start the assignment!

In [1]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
import cv2              # opencv-python
import ffmpeg           # ffmpeg-python
import subprocess   # for calling local executables such as ffmpeg.exe
import pandas as pd  # pandas
from fractions import Fraction as frac # simplifying fractions

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [498]:
# Place your own imports here.
import IPython
from IPython.core.display import HTML
import json
import glob

In [183]:
# Place any helper functions, global variables and classes here.

# For example:the function you need to play back a mp4-video file.
# You may use this function to display the videos in your definition file during the assessment for demoing the solutions.
def VideoPlayer(videoFile: str) -> None:
    IPython.display.display(
        HTML("""
            <video alt="test" controls>
            <source src={} type="video/mp4">
            </video>
        """.format(videoFile))
    )
    
    return

# function to play audio files
def AudioPlayer(audioFile: str) -> None:
    IPython.display.display(IPython.display.Audio(
        audioFile
    ))
    
    return

In [143]:
VideoPlayer("./resources/video/sample1.mp4")


## Task 3.1: Create a slide show (Video) from multiple images and convert it to greyscale 

In [161]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
def CreateVideoFromImages(inImgLibFolder: str, imageFormat: str, durationInSec: int, convertToGreyScale: bool, outFolder: str, outVideo: str) -> None:
    
    if(convertToGreyScale):
        subprocess.run("ffmpeg -y -framerate 1/"+ str(durationInSec) +" -pattern_type glob -i '"+ inImgLibFolder +"*"+ imageFormat +"' -vf \"format=gray, scale=1280:720\" -c:v libx264 -pix_fmt yuv420p -shortest " + outFolder + outVideo + ".mp4", shell=True)
    else:
        subprocess.run("ffmpeg -y -framerate 1/"+ str(durationInSec) +" -pattern_type glob -i '"+ inImgLibFolder +"*"+ imageFormat +"' -vf \"scale=1280:720\" -c:v libx264 -pix_fmt yuv420p -shortest " + outFolder + outVideo + ".mp4", shell=True)

    # play video
    VideoPlayer(outFolder + outVideo + ".mp4")


In [162]:
# Test your function here.
CreateVideoFromImages("./resources/images/Task3.1/", ".jpg", 2, True, "./resources/images/Task3.1/", "slideshow")

## Task 3.2: Extract the Audio Track from a Video File 

In [254]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.

def SplitAudioVideoTracks(inVideo: str, outFolder: str, outVideoTrack: str, outAudioTrack: str) -> None:
    
    # check if video has audio
    hasAudio = False

    getStreams = subprocess.run("ffprobe -i '"+ inVideo +"' -show_streams -select_streams a -loglevel error", shell=True, capture_output=True)
    
    if(getStreams.stdout):
        hasAudio = True
    else:
        hasAudio = False
        print("input video has no sound")
        return
        
    # https://ffmpeg.org/ffmpeg.html#Audio-Options
    subprocess.run("ffmpeg -i "+ inVideo +" -vf \"scale=1280:720\" -c copy -an " + outFolder + outVideoTrack, shell=True)
    
    # simpy save the .mp4 as .mp3
    subprocess.run("ffmpeg -i "+ inVideo + " " + outFolder + outAudioTrack, shell=True)
    
    # play video & audio
    VideoPlayer("./resources/video/Task3.2/noSoundVideo.mp4")
    AudioPlayer("./resources/video/Task3.2/noVideoSound.mp3")

In [256]:
# Test your function here.
SplitAudioVideoTracks("./resources/video/Task3.2/sample1.mp4", "./resources/video/Task3.2/", "noSoundVideo.mp4", "noVideoSound.mp3")

## Task 3.3: Replace the Audio Track in a Video File

In [278]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.

def AddOrReplaceAudio(inVideo: str, inAudio: str, outFolder: str, outVideo: str) -> None:
    
    # check if video contains audio
    hasAudio = False

    getStreams = subprocess.run("ffprobe -i '"+ inVideo +"' -show_streams -select_streams a -loglevel error", shell=True, capture_output=True)
    
    if(getStreams.stdout):
        hasAudio = True
    else:
        hasAudio = False
        
    if(hasAudio):
        subprocess.run("ffmpeg -y -i "+ inVideo +" -i '"+ inAudio + "' -map 0:v -map 1:a -c:v copy -shortest " + outFolder + outVideo + ".mp4", shell=True)
    else:
        subprocess.run("ffmpeg -y -i "+ inVideo +" -i '"+ inAudio + "' -map 0:v -map 1:a -c:v copy -shortest " + outFolder + outVideo + ".mp4", shell=True)

    VideoPlayer(outFolder + outVideo + ".mp4")

In [279]:
# Test your function here.
# add audio
AddOrReplaceAudio("./resources/video/Task3.3/noSoundVideo.mp4", "./resources/video/Task3.3/DashTheCurry[Skit].mp3", "./resources/video/Task3.3/", "modifiedAudioVideo")
# replace audio
AddOrReplaceAudio("./resources/video/Task3.3/sample2.mp4", "./resources/video/Task3.3/DashTheCurry[Skit].mp3", "./resources/video/Task3.3/", "modifiedAudioVideo")


## Task 3.4: Combine Videos

In [496]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification.
def CombineVideos(inVideoLibFolder: str, outFolder: str, outVideo: str) -> None:
    pass

# Replace the parameters and return type of the following function according to the task specification.
def VideoMetadataExtractor(inputDir: str) -> pd.DataFrame():
    columns = [ 'vCodec',
                'vCodecID',
                'vDur',
                'vFPS',
                'vHeight',
                'vWidth',
                'aspectRatio',
                'aCodec',
                'aCodecID',
                'aChannels',
                'aSampleRate',
                'aBitRate' ]
    
    directory = glob.glob(inputDir)
    df = pd.DataFrame()
    
    rows = []
    
    for file in directory:
        with open(file, 'rb') as fl:
            
            rows.append(fl.name)
            
            # get format data as json
            formatData = subprocess.run("ffprobe -v error -hide_banner -of default=noprint_wrappers=0 -print_format json -select_streams v:0 -show_format " + fl.name, shell=True, capture_output=True)
            jsonFormatData = json.loads(formatData.stdout)

            # get video stream data as json
            streamVideoData = subprocess.run("ffprobe -v error -hide_banner -of default=noprint_wrappers=0 -print_format json -select_streams v:0 -show_streams " + fl.name, shell=True, capture_output=True)
            jsonStreamVideoData = json.loads(streamVideoData.stdout)

            # get audio stream data as json
            streamAudioData = subprocess.run("ffprobe -v error -hide_banner -of default=noprint_wrappers=0 -print_format json -select_streams a:0 -show_streams " + fl.name, shell=True, capture_output=True)
            jsonStreamAudioData = json.loads(streamAudioData.stdout)
           
            # extract metadata from json
            vCodec = jsonStreamVideoData['streams'][0]['codec_name']
            vCodecID = jsonStreamVideoData['streams'][0]['codec_tag']
            vDur = jsonStreamVideoData['streams'][0]['duration']
            vFPS = jsonStreamVideoData['streams'][0]['r_frame_rate'] 
            vHeight = jsonStreamVideoData['streams'][0]['height']
            vWidth = jsonStreamVideoData['streams'][0]['width']
            aspectRatio = str(vWidth) + ":" + str(vHeight)
            aCodec = jsonStreamAudioData['streams'][0]['codec_name'] 
            aCodecID = jsonStreamAudioData['streams'][0]['codec_tag']  
            aChannels = jsonStreamAudioData['streams'][0]['channels'] 
            aSampleRate = jsonStreamAudioData['streams'][0]['sample_rate'] 
            aBitRate = jsonStreamAudioData['streams'][0]['bit_rate'] 
            
            # append new dataFrame to initial dataFrame
            newRow = [ vCodec, vCodecID,vDur,vFPS,vHeight,vWidth,aspectRatio,aCodec,aCodecID,aChannels,aSampleRate,aBitRate ]
            
            # append new dataFrame to initial dataFrame]
            df2 = pd.DataFrame(data = [newRow],columns = columns)
            df = df.append(df2 ,ignore_index = True) 

    # set index column rows + name
    df.index = rows
    df.index.name = "filename"
    
    return df

In [497]:
# Test your function here.
VideoMetadataExtractor("./resources/video/Task3.4/*")

,vCodec,vCodecID,vDur,vFPS,vHeight,vWidth,aspectRatio,aCodec,aCodecID,aChannels,aSampleRate,aBitRate
filename,,,,,,,,,,,,
./resources/video/Task3.4/sample2.mp4,h264,0x31637661,7.107100,30000/1001,480,640,640:480,aac,0x6134706d,2,44100,128289
./resources/video/Task3.4/sample1.mp4,h264,0x31637661,42.375667,30000/1001,240,352,352:240,aac,0x6134706d,2,44100,127761
./resources/video/Task3.4/sample3.avi,h264,0x34363248,7.173833,24000/1001,486,720,720:486,aac,0x00ff,2,44100,128000


## Task 3.5: Blend an Image in a Video

In [ ]:
# Write your code here!

# Replace the parameters and return type of the following function according to the task specification.
def AddFadingImage(...) -> None: 
    pass

In [ ]:
# Test your function here.

## Task 3.6: Create a Video Collage - blend two videos into one

In [ ]:
# Write your function here.

# Replace the parameters and return type of the following function according to the task specification. 
def VideoClipMixer(...) -> None: 
    pass


In [ ]:
# Test your function here.